In [1]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns
# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report

In [2]:
from keras.utils.np_utils import to_categorical

In [3]:
df = pd.read_csv('FIFA_data.csv')
df

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18201,18201,243413,D. Walsh,18,https://cdn.sofifa.org/players/4/19/243413.png,Republic of Ireland,https://cdn.sofifa.org/flags/25.png,47,68,Waterford FC,...,43.0,44.0,47.0,53.0,9.0,10.0,9.0,11.0,13.0,€153K
18202,18202,238813,J. Lundstram,19,https://cdn.sofifa.org/players/4/19/238813.png,England,https://cdn.sofifa.org/flags/14.png,47,65,Crewe Alexandra,...,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,€143K
18203,18203,243165,N. Christoffersson,19,https://cdn.sofifa.org/players/4/19/243165.png,Sweden,https://cdn.sofifa.org/flags/46.png,47,63,Trelleborgs FF,...,42.0,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0,€113K
18204,18204,241638,B. Worman,16,https://cdn.sofifa.org/players/4/19/241638.png,England,https://cdn.sofifa.org/flags/14.png,47,67,Cambridge United,...,41.0,32.0,13.0,11.0,6.0,5.0,10.0,6.0,13.0,€165K


In [4]:
na = pd.notnull(df["Position"])
df = df[na]
#df["Position"].unique()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18146 entries, 0 to 18205
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                18146 non-null  int64  
 1   ID                        18146 non-null  int64  
 2   Name                      18146 non-null  object 
 3   Age                       18146 non-null  int64  
 4   Photo                     18146 non-null  object 
 5   Nationality               18146 non-null  object 
 6   Flag                      18146 non-null  object 
 7   Overall                   18146 non-null  int64  
 8   Potential                 18146 non-null  int64  
 9   Club                      17917 non-null  object 
 10  Club Logo                 18146 non-null  object 
 11  Value                     18146 non-null  object 
 12  Wage                      18146 non-null  object 
 13  Special                   18146 non-null  int64  
 14  Prefer

In [6]:
df = df[["Position", 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes']]

In [7]:
forward_player = ["ST", "LW", "RW", "LF", "RF", "RS","LS", "CF"]
midfielder_player = ["CM","RCM","LCM", "CDM","RDM","LDM", "CAM", "LAM", "RAM", "RM", "LM"]
defender_player = ["CB", "RCB", "LCB", "LWB", "RWB", "LB", "RB"]

In [8]:
df.loc[df["Position"] == "GK", "Position"] = 0

df.loc[df["Position"].isin(defender_player), "Position"] = 1

df.loc[df["Position"].isin(midfielder_player), "Position"] = 2

df.loc[df["Position"].isin(forward_player), "Position"] = 3

In [9]:
x = df.drop("Position", axis = 1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = pd.DataFrame(sc.fit_transform(x))
y = df["Position"]
y[:10]

0    3
1    3
2    3
3    0
4    2
5    3
6    2
7    3
8    1
9    0
Name: Position, dtype: object

In [10]:
y_cat = to_categorical(y)
y_cat[:10]

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2)

In [12]:
model = Sequential()
model.add(Dense(60, input_shape = (33,), activation = "relu"))
model.add(Dense(15, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(4, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                2040      
                                                                 
 dense_1 (Dense)             (None, 15)                915       
                                                                 
 dropout (Dropout)           (None, 15)                0         
                                                                 
 dense_2 (Dense)             (None, 4)                 64        
                                                                 
Total params: 3,019
Trainable params: 3,019
Non-trainable params: 0
_________________________________________________________________


2023-02-08 14:52:58.894007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/shreyanthhg/opt/anaconda3/envs/Tensorflow/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [13]:
model.fit(x_train, y_train, verbose=1,validation_split=0.33, epochs=10)

Epoch 1/10
304/304 [==============================] - 1s 1ms/step - loss: 0.3985 - accuracy: 0.8366 - val_loss: 0.3169 - val_accuracy: 0.8650
Epoch 2/10
304/304 [==============================] - 0s 783us/step - loss: 0.3414 - accuracy: 0.8613 - val_loss: 0.3025 - val_accuracy: 0.8758
Epoch 3/10
304/304 [==============================] - 0s 823us/step - loss: 0.3301 - accuracy: 0.8660 - val_loss: 0.2962 - val_accuracy: 0.8794
Epoch 4/10
304/304 [==============================] - 0s 822us/step - loss: 0.3173 - accuracy: 0.8687 - val_loss: 0.3057 - val_accuracy: 0.8721
Epoch 5/10
304/304 [==============================] - 0s 835us/step - loss: 0.3193 - accuracy: 0.8725 - val_loss: 0.3034 - val_accuracy: 0.8825
Epoch 6/10
304/304 [==============================] - 0s 772us/step - loss: 0.3139 - accuracy: 0.8681 - val_loss: 0.3019 - val_accuracy: 0.8823
Epoch 7/10
304/304 [==============================] - 0s 773us/step - loss: 0.3136 - accuracy: 0.8725 - val_loss: 0.3001 - val_accuracy: 0

In [14]:
predict_x=model.predict(x_test) 
print(predict_x)
classes_x=np.argmax(predict_x,axis=1)
print(classes_x)
from sklearn.metrics import confusion_matrix
y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)
confusion_matrix(y_test_class, classes_x)

114/114 [==============================] - 0s 389us/step
[[5.21246989e-07 3.30737889e-01 6.66231334e-01 3.03025544e-03]
 [2.83658630e-09 6.22731401e-03 9.90929544e-01 2.84309918e-03]
 [1.19173819e-06 9.27792280e-05 6.80454373e-01 3.19451660e-01]
 ...
 [9.13133480e-09 4.54500196e-06 6.24216914e-01 3.75778586e-01]
 [5.08914653e-08 4.75871097e-03 9.81250346e-01 1.39909862e-02]
 [2.53487877e-23 1.26030375e-11 5.44703798e-03 9.94553030e-01]]
[2 2 2 ... 2 2 3]
114/114 [==============================] - 0s 356us/step


array([[ 373,    0,    0,    0],
       [   0, 1115,  116,    1],
       [   0,   81, 1233,   33],
       [   0,    4,  249,  425]])

In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_test_class, classes_x))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       373
           1       0.93      0.91      0.92      1232
           2       0.77      0.92      0.84      1347
           3       0.93      0.63      0.75       678

    accuracy                           0.87      3630
   macro avg       0.91      0.86      0.88      3630
weighted avg       0.88      0.87      0.86      3630

